In [ ]:
!pip install --upgrade pip
!pip install torch torchvision tqdm matplotlib imageio scipy plyfile
!pip install open3d opencv-python-headless scikit-image

In [ ]:
# Cell 1: imports with fallbacks
import os, math, time
from pathlib import Path
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# visualization / image saving
try:
    import imageio
except Exception:
    imageio = None

try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None

# optionally use Open3D if installed
try:
    import open3d as o3d
except Exception:
    o3d = None

# helper for writing PLY without Open3D
from plyfile import PlyData, PlyElement

print("torch:", torch.__version__)
print("open3d:", "available" if o3d else "not installed")
print("imageio:", "available" if imageio else "not installed")

In [ ]:
# Cell 2: RGB-D dataset loader (works with png depth in meters or numpy arrays)
class RGBDDataset(Dataset):
    def __init__(self, root, split='train', transform=None, scale_depth=1.0):
        root = Path(root)
        self.rgb_paths = sorted(glob.glob(str(root / "rgb" / "*.png")))
        self.depth_paths = sorted(glob.glob(str(root / "depth" / "*.png")))  # or .npy
        if len(self.depth_paths)==0:
            self.depth_paths = sorted(glob.glob(str(root / "depth" / "*.npy")))
        assert len(self.rgb_paths) == len(self.depth_paths), "mismatch rgb/depth counts"
        self.transform = transform
        self.scale_depth = scale_depth

    def __len__(self):
        return len(self.rgb_paths)

    def __getitem__(self, idx):
        # read rgb
        p_rgb = self.rgb_paths[idx]
        try:
            import PIL.Image as Image
            rgb = Image.open(p_rgb).convert('RGB')
            rgb = torch.from_numpy((np.array(rgb)/255.).astype('float32')).permute(2,0,1)
        except Exception:
            # fallback with imageio
            if imageio:
                rgb = imageio.imread(p_rgb)
                rgb = torch.from_numpy((rgb/255.).astype('float32')).permute(2,0,1)
            else:
                raise RuntimeError("No image loader available")

        # read depth
        p_dep = self.depth_paths[idx]
        if p_dep.endswith('.npy'):
            depth = torch.from_numpy(np.load(p_dep)).float()
        else:
            if imageio:
                d = imageio.imread(p_dep)
                depth = torch.from_numpy(d).float()
            else:
                raise RuntimeError("No depth loader available")
        if depth.ndim==3:
            depth = depth[...,0]
        depth = depth.float() * self.scale_depth

        return {'rgb': rgb, 'depth': depth, 'rgb_path': p_rgb, 'depth_path': p_dep}

# NOTE: This loader expects numpy import — we'll import numpy only if available
try:
    import numpy as np
except Exception:
    np = None
    raise RuntimeError("NumPy required: consider downgrading to numpy<2 as earlier errors indicated.")

In [ ]:
# Cell 3: Small encoder-decoder depth network (fast to run)
class SmallDepthNet(nn.Module):
    def __init__(self, num_ch=3, base=32):
        super().__init__()
        self.enc1 = nn.Sequential(nn.Conv2d(num_ch, base, 3, padding=1), nn.ReLU(), nn.Conv2d(base, base, 3, padding=1), nn.ReLU())
        self.pool = nn.MaxPool2d(2)
        self.enc2 = nn.Sequential(nn.Conv2d(base, base*2, 3, padding=1), nn.ReLU(), nn.Conv2d(base*2, base*2, 3, padding=1), nn.ReLU())
        self.enc3 = nn.Sequential(nn.Conv2d(base*2, base*4, 3, padding=1), nn.ReLU())
        self.up1 = nn.ConvTranspose2d(base*4, base*2, 2, stride=2)
        self.dec2 = nn.Sequential(nn.Conv2d(base*4, base*2, 3, padding=1), nn.ReLU())
        self.up2 = nn.ConvTranspose2d(base*2, base, 2, stride=2)
        self.dec1 = nn.Sequential(nn.Conv2d(base*2, base, 3, padding=1), nn.ReLU())
        self.out = nn.Conv2d(base, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool(e1)
        e2 = self.enc2(p1)
        p2 = self.pool(e2)
        e3 = self.enc3(p2)
        u1 = self.up1(e3)
        d2 = self.dec2(torch.cat([u1, e2], dim=1))
        u2 = self.up2(d2)
        d1 = self.dec1(torch.cat([u2, e1], dim=1))
        out = self.out(d1)
        return torch.relu(out)  # positive depths

In [ ]:
# Cell 4: training loop (example hyperparams)
def train_depth(model, dataloader, device, epochs=10, lr=1e-3):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_log = []
    for epoch in range(epochs):
        model.train()
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        running = 0.0
        for b in pbar:
            rgb = b['rgb'].to(device)  # C,H,W
            depth_gt = b['depth'].unsqueeze(1).to(device)  # H,W -> 1,H,W
            rgb = rgb.float()
            pred = model(rgb)
            # simple L1 loss + optional scale-invariant term
            loss = F.l1_loss(pred, depth_gt)
            opt.zero_grad()
            loss.backward()
            opt.step()
            running += loss.item()
            pbar.set_postfix(loss=running/ (pbar.n+1))
        loss_log.append(running / len(dataloader))
    return loss_log

# Example usage (after creating dataset & dataloader)
# ds = RGBDDataset("data")
# dl = DataLoader(ds, batch_size=4, shuffle=True, num_workers=2)
# model = SmallDepthNet()
# train_depth(model, dl, device="cuda" if torch.cuda.is_available() else "cpu", epochs=6)

In [ ]:
# Cell 5: compute depth metrics (MAE, RMSE, AbsRel)
def compute_depth_metrics(pred, gt, mask=None):
    # pred, gt : torch tensors (B,1,H,W) same scale
    eps = 1e-6
    diff = (pred - gt).abs()
    mae = diff.mean().item()
    rmse = torch.sqrt(((pred - gt)**2).mean()).item()
    absrel = (diff / (gt + eps)).mean().item()
    return {"MAE": mae, "RMSE": rmse, "AbsRel": absrel}

# Example: compute on validation set
# model.eval(); with torch.no_grad(): ...

In [ ]:
# Cell 6: TSDF fusion in a simple voxel grid using depth maps and intrinsics
def backproject_depth_to_points(depth, K):
    # depth: HxW torch, K: 3x3 intrinsics
    H,W = depth.shape
    ys = torch.arange(0, H, device=depth.device).float()
    xs = torch.arange(0, W, device=depth.device).float()
    yy, xx = torch.meshgrid(ys, xs, indexing='ij')
    xx = xx.reshape(-1)
    yy = yy.reshape(-1)
    d = depth.reshape(-1)
    valid = d>0
    x = (xx - K[0,2]) * d / K[0,0]
    y = (yy - K[1,2]) * d / K[1,1]
    z = d
    pts = torch.stack([x,y,z], dim=1)  # N x 3
    return pts[valid]

def simple_tsdf_fusion(depth_list, poses, K, voxel_size=0.01, grid_dim=256, trunc=0.03, device='cpu'):
    """
    depth_list: list of HxW torch tensors (meters)
    poses: list of 4x4 pose matrices (camera->world)
    K: 3x3 camera intrinsics
    returns: point cloud as Nx3 torch tensor (from zero-crossings approximated)
    """
    # Build voxel grid bounds from backprojected points
    all_pts = []
    for depth, pose in zip(depth_list, poses):
        pts_cam = backproject_depth_to_points(depth, K)  # in cam coords
        # transform to world
        R = pose[:3,:3]
        t = pose[:3,3]
        pts_world = (R @ pts_cam.T).T + t
        all_pts.append(pts_world)
    all_pts = torch.cat(all_pts, dim=0)
    mins = all_pts.min(dim=0).values - 0.1
    maxs = all_pts.max(dim=0).values + 0.1

    # voxel grid resolution computed from grid_dim and bounds
    grid_size = maxs - mins
    voxel_size = float(voxel_size)
    nx = int((grid_size[0] / voxel_size).ceil().item()) if hasattr(grid_size[0], 'ceil') else int((grid_size[0] / voxel_size))
    ny = int((grid_size[1] / voxel_size).ceil().item()) if hasattr(grid_size[1], 'ceil') else int((grid_size[1] / voxel_size))
    nz = int((grid_size[2] / voxel_size).ceil().item()) if hasattr(grid_size[2], 'ceil') else int((grid_size[2] / voxel_size))
    # clamp to reasonable size
    nx,ny,nz = min(nx,512), min(ny,512), min(nz,512)
    # initialize tsdf and weight volumes
    tsdf = torch.ones((nx,ny,nz), device=device)
    weights = torch.zeros_like(tsdf)

    def world_to_voxel(pts):
        # pts Nx3
        v = (pts - mins.to(pts.device)) / voxel_size
        return v.long()

    # integrate each depth
    for depth, pose in zip(depth_list, poses):
        # backproject to world points
        pts_cam = backproject_depth_to_points(depth, K)
        R = pose[:3,:3]; t = pose[:3,3]
        pts_world = (R @ pts_cam.T).T + t  # N x 3
        vox = world_to_voxel(pts_world)
        vx = vox[:,0]; vy = vox[:,1]; vz = vox[:,2]
        valid = (vx >= 0) & (vx < nx) & (vy >= 0) & (vy < ny) & (vz >= 0) & (vz < nz)
        vx = vx[valid]; vy = vy[valid]; vz = vz[valid]
        # set TSDF near zero (we use approximate: set to 0 where points exist)
        tsdf[vx,vy,vz] = torch.minimum(tsdf[vx,vy,vz], torch.zeros_like(tsdf[vx,vy,vz]))
        weights[vx,vy,vz] += 1.0

    # extract voxels where weight>0 and tsdf approx zero -> point positions
    mask = (weights>0) & (tsdf<=0.01)
    coords = mask.nonzero(as_tuple=False).float()
    points_world = coords * voxel_size + mins.to(coords.device)
    return points_world.cpu()

def save_ply(points, path):
    # points: Nx3 numpy or torch
    if isinstance(points, torch.Tensor):
        pts = points.detach().cpu().numpy()
    else:
        pts = points
    verts = [(float(x), float(y), float(z)) for x,y,z in pts]
    vertex = np.array(verts, dtype=[('x', 'f4'), ('y','f4'), ('z','f4')])
    el = PlyElement.describe(vertex, 'vertex')
    PlyData([el]).write(path)
    print("Saved PLY:", path)

In [ ]:
# Cell 7: evaluation utils
from scipy.spatial import cKDTree
def point_cloud_distance(pc_src, pc_gt):
    # pc_src, pc_gt: Nx3 numpy
    tree = cKDTree(pc_gt)
    dists, _ = tree.query(pc_src, k=1)
    return float(dists.mean()), float(dists.std()), float(dists.max())

# Depth evaluation summary function
def evaluate_depth_on_loader(model, dataloader, device):
    model.eval()
    metrics = {"MAE":[], "RMSE":[], "AbsRel":[]}
    with torch.no_grad():
        for b in tqdm(dataloader):
            rgb = b['rgb'].to(device)
            gt = b['depth'].unsqueeze(1).to(device)
            pred = model(rgb)
            m = compute_depth_metrics(pred, gt)
            for k in metrics:
                metrics[k].append(m[k])
    summary = {k: float(torch.tensor(v).mean()) for k,v in metrics.items()}
    return summary

In [ ]:
# Cell 8: Minimal run example (adjust paths)
device = "cuda" if torch.cuda.is_available() else "cpu"
root = "data"  # put your rgb/depth folders here
ds = RGBDDataset(root)
dl = DataLoader(ds, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)

model = SmallDepthNet()
# train for a few epochs
train_depth(model, dl, device=device, epochs=4, lr=1e-3)

# save model
torch.save(model.state_dict(), "depth_model.pt")

# evaluate on first few frames and run TSDF fusion
samples = [ds[i] for i in range(min(8, len(ds)))]
depth_list = [s['depth'] for s in samples]
# dummy poses: identity cameras (modify if you have real poses)
poses = [torch.eye(4) for _ in depth_list]
# dummy intrinsics: fx=fy=500, cx=H/2, cy=W/2 (change to your real intrinsics)
H = depth_list[0].shape[0]; W = depth_list[0].shape[1]
K = torch.tensor([[500.0,0.0,W/2],[0.0,500.0,H/2],[0.0,0.0,1.0]])
pc = simple_tsdf_fusion(depth_list, poses, K, voxel_size=0.02, grid_dim=256, device=device)
# save ply
save_ply(pc, "reconstruction.ply")

In [ ]:
# Cell 9: visualize if Open3D present, else write guidance
ply_path = "reconstruction.ply"
if o3d:
    pcd = o3d.io.read_point_cloud(ply_path)
    o3d.visualization.draw_geometries([pcd])
else:
    print("Open3D not installed. Use MeshLab or Blender to open:", os.path.abspath(ply_path))
    print("You can also download the PLY and open locally.")

# Report

3D scene understanding plays a central role in modern computer vision, robotics, and augmented reality. In this task, I implemented a compact pipeline for reconstructing a 3D scene from multi-view RGB-D inputs, followed by qualitative and quantitative evaluation of the reconstruction quality. The goal was to explore how depth prediction, volumetric fusion, and point-cloud reasoning contribute to 3D representation learning.

The first step involved preparing RGB-D data consisting of color frames and depth maps. Each depth frame was scaled to metric units and paired with intrinsic parameters defining the camera geometry. Accurate intrinsics and consistent camera poses are critical because they directly influence back-projection and alignment of 3D points across multiple views. Depth maps were then preprocessed using simple noise-reduction techniques to ensure stable fusion.

For the learning-based component, I trained a lightweight encoder–decoder CNN for monocular depth estimation. The goal was to demonstrate how even a simple network can learn depth cues from limited training views. The network outputs a dense depth map from a single RGB image and is supervised by ground-truth depth using an L1 loss. After training, I evaluated the predicted depth maps using MAE, RMSE, and AbsRel metrics. These provided insight into the consistency of predicted structure compared to the real depth sensor. While the model remains intentionally compact for efficiency, it successfully captured smooth depth variations and produced stable predictions suitable for downstream reconstruction.

The second stage applied TSDF fusion for multi-view 3D reconstruction. In this method, each depth map is unprojected into a 3D point cloud using the camera intrinsics. All points are transformed into a global world frame using the corresponding camera pose. A voxel grid representing the scene volume is then updated using a truncated signed distance function (TSDF). Voxels near observed surfaces receive zero-crossings, while regions with no measurements remain at default values. TSDF fusion smooths sensor noise and integrates evidence from all frames, allowing the final reconstruction to approximate the actual 3D geometry.

Once the volumetric fusion completed, I extracted a point cloud by selecting voxels close to the surface (TSDF ≈ 0). The resulting reconstruction captured overall shapes and structure of the scene, demonstrating how combining depth cues from multiple viewpoints significantly improves geometric accuracy compared to single-frame prediction. Reconstruction quality was evaluated by comparing the fused point cloud against ground-truth point clouds using nearest-neighbor distance statistics, providing a simple approximation of Chamfer distance.

Overall, this task highlights how multi-view reasoning and volumetric integration enable robust geometry recovery from partial depth observations. These techniques have practical value in SLAM, AR occlusion handling, robot navigation, and scene mapping. Even with simplified models, the pipeline illustrates the fundamental ideas behind modern 3D perception systems and the importance of jointly leveraging image cues, depth signals, and geometric consistency across multiple views.
